base code borrowed from [this Google Colab Notebook](https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb).

Refactored by [Shuyi Wang](https://www.linkedin.com/in/shuyi-wang-b3955026/) and [Yan Sun](https://github.com/SunYanCN)

Please refer to [this Medium Article](https://medium.com/@wshuyi/how-to-do-text-binary-classification-with-bert-f1348a25d905) for detailed information.



In [1]:
!pip install bert-text

In [2]:
from bert_text import run_on_dfs

W0723 19:18:09.785237 140061423093632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0723 19:18:09.788282 140061423093632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert_text/bert_text.py:10: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0723 19:18:09.789436 140061423093632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert_text/bert_text.py:10: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.



In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [4]:
!wget https://raw.githubusercontent.com/BorisVelichkov/paper/master/dataset/interviews.csv -P "dataset"

--2019-07-23 19:18:16--  https://raw.githubusercontent.com/BorisVelichkov/paper/master/dataset/interviews.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71382 (70K) [text/plain]
Saving to: ‘dataset/interviews.csv’

interviews.csv      100%[===================>]  69.71K  --.-KB/s    in 0.02s   

2019-07-23 19:18:16 (3.37 MB/s) - ‘dataset/interviews.csv’ saved [71382/71382]



In [5]:
df = pd.read_csv('dataset/interviews.csv', index_col=0)
df.head()

,file_name,label,text
0,BG_Box_AleksandarPovetkin_vs_AnthonyDjoshua_NO,NO,"Аз съм в много добра форма, проведох отличен т..."
1,BG_Box_AndiRuiz_vs_AnthonyDjoshua_YES,YES,"Всички ме подценяват, но това ще бъде страшна ..."
2,BG_Box_AnthonyDjoshua_vs_AleksandarPovetkin_YES,YES,"Ако оставим настрана техниката и качествата, и..."
3,BG_Box_AnthonyDjoshua_vs_AndiRuiz_NO,NO,"Приемам мача с Анди напълно сериозно, изобщо н..."
4,BG_Box_AntonyDjoshua_vs_VladimirKlichko_YES,YES,"Бях изтласкан на места, където никога не съм б..."


In [6]:
df.drop('file_name', axis=1, inplace=True)

le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

df.head()

,label,text
0,0,"Аз съм в много добра форма, проведох отличен т..."
1,1,"Всички ме подценяват, но това ще бъде страшна ..."
2,1,"Ако оставим настрана техниката и качествата, и..."
3,0,"Приемам мача с Анди напълно сериозно, изобщо н..."
4,1,"Бях изтласкан на места, където никога не съм б..."


In [0]:
def gen_train_and_test(df, test_size=0.20, random_state=42):
    X = df.loc[:, df.columns != 'label']
    y = df.label
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    train = X_train.join(y_train)
    test = X_test.join(y_test)
    
    return train, test

In [0]:
train, test = gen_train_and_test(df)

In [9]:
train.head()

,text,label
12,Важно е да си концентриран и целенасочен в тов...,1
4,"Бях изтласкан на места, където никога не съм б...",1
37,Сега оценявам и се радвам на всяка една победа...,0
8,С нетърпение очаквам битката с Кубрат Пулев.\r...,0
3,"Приемам мача с Анди напълно сериозно, изобщо н...",0


In [10]:
test.head()

,text,label
13,"Статутът ми на фаворит не ми влияе, почти няма...",1
39,Във всеки един мач трябва да си подготвен.\r\n...,1
30,В Монте Карло изиграх добър мач срещу Димитров...,1
45,Той няма как да не вкара всяка топка в корта в...,0
17,"Познавам го, голям и здрав е, бил се е с добри...",1


In [0]:
myparam = {
    "DATA_COLUMN": "text",
    "LABEL_COLUMN": "label",
    "LEARNING_RATE": 2e-5,
    "NUM_TRAIN_EPOCHS": 100,
    "bert_model_hub":"https://tfhub.dev/google/bert_multi_cased_L-12_H-768_A-12/1"
}

In [0]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)

In [13]:
result, estimator = run_on_dfs(train, test, **myparam)

I0723 19:19:17.190720 140061423093632 saver.py:1499] Saver not created because there are no variables in the graph to restore
W0723 19:19:17.321282 140061423093632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert_text/bert_text.py:19: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0723 19:19:18.270442 140061423093632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0723 19:19:18.691276 140061423093632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/run_classifier.py:774: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

I0723 19:19:18.692321 140061423093632 run_classifier.py:774] Writing example 0 of 40
I0723 19:19:18.697306 140061423093632 run_classifier.py:461] *** Example ***
I0723 19:19:18.698803 140061423093632 run_classifier.py:462

In [14]:
result

{'auc': 0.625,
 'eval_accuracy': 0.7,
 'f1_score': 0.57142854,
 'false_negatives': 3.0,
 'false_positives': 0.0,
 'global_step': 125,
 'loss': 2.799212,
 'precision': 1.0,
 'recall': 0.25,
 'true_negatives': 6.0,
 'true_positives': 1.0}